In [473]:
%load_ext autoreload
%autoreload 2

In [342]:
# to get JSON files from TOS API
import urllib, json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [119]:
#create a list of file names to pull each json file
from os import listdir
from os.path import isfile, join
companies = [f for f in listdir('tosdr.org/api/1/service') if isfile(join('tosdr.org/api/1/service', f))]

In [415]:
# nlp imports, chose TFIDF since some words are repeated
from sklearn.feature_extraction.text import TfidfVectorizer

In [244]:
#Pull out the data 
data_list = []

for company in companies:
    with open(f'tosdr.org/api/1/service/{company}') as json_data:
        data_list.append(json.load(json_data))

In [553]:
#Create main dataframe, drop unnecessary columns, expand list of lists

pd_list = []
for data in data_list:
    pd_list.append(pd.DataFrame.from_dict(data['pointsData']).T.loc[:,['id','quoteText','services','title','tosdr']])

alldata_df = pd.concat(pd_list,axis=0).reset_index()
todsr_df = json_normalize(alldata_df['tosdr'])
df = pd.concat((alldata_df,todsr_df),axis=1)
df = df.drop('tosdr',axis=1).explode('services')

In [557]:
#create master document column with tldr as base and quote text as secondar

df['document'] = df['tldr']
df['document'] = np.where(df['document'] == '',df['quoteText'],df['document'])
df['document'] = np.where(df['document'] == 'Generated through the annotate view',df['quoteText'],df['document'])
df['document'].fillna(df['quoteText'])

#if there is no text in the tldr or quoteText column, drop and reset index
df.dropna(axis=0,subset=['document'],inplace=True)
df = df.reset_index(drop=True)

In [558]:
df

,index,id,quoteText,services,title,binding,case,point,privacyRelated,score,tldr,sources,irrelevant,reason,tmp_rating,document
0,4411,4411,"Signal does not sell, rent or monetize your pe...",signal,This service does not sell your personal data,True,This service does not sell your personal data,good,True,25,"The terms of service state that ""Signal does n...",NaN,NaN,NaN,NaN,"The terms of service state that ""Signal does n..."
1,4412,4412,You must be at least 13 years old to use our S...,signal,You must be at least 13 years old to use the s...,True,This service is only available to users of a c...,neutral,NaN,15,"The terms of service state: ""You must be at le...",NaN,NaN,NaN,NaN,"The terms of service state: ""You must be at le..."
2,4414,4414,You agree to resolve any Claim you have with u...,signal,Court of law is in California,True,The court of law governing the terms is in loc...,neutral,NaN,0,"The terms of service state: ""You agree to reso...",NaN,NaN,NaN,NaN,"The terms of service state: ""You agree to reso..."
3,4415,4415,"We may modify, suspend, or terminate your acce...",signal,The service can delete your account without pr...,True,The service can delete your account without pr...,bad,NaN,60,"The terms of service state: ""We may modify, su...",NaN,NaN,NaN,NaN,"The terms of service state: ""We may modify, su..."
4,4416,4416,We work with third parties to provide some of ...,signal,Third parties may be involved in operating the...,True,Third parties may be involved in operating the...,bad,NaN,15,"The privacy policy states: ""We work with third...",NaN,NaN,NaN,NaN,"The privacy policy states: ""We work with third..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,4523,4523,NaN,tos;dr,The terms for ToS;DR are easy to read,True,The terms for this service are easy to read,good,NaN,15,"Altogether, ToS and PP only make up eight line...",NaN,NaN,NaN,NaN,"Altogether, ToS and PP only make up eight line..."
2556,4524,4524,Nothing here should be considered legal advice...,tos;dr,ToS;DR gives no guarantee regarding quality,True,This services gives no guarantee regarding qua...,bad,NaN,20,"The ToS state: ""Nothing here should be conside...",NaN,NaN,NaN,NaN,"The ToS state: ""Nothing here should be conside..."
2557,7679,7679,We do not use cookies or any other tracking te...,tos;dr,This service does not track you,True,This service does not track you,good,NaN,100,Generated through the annotate view,NaN,NaN,NaN,NaN,We do not use cookies or any other tracking te...
2558,7683,7683,By contributing to this project (e.g.\nby send...,tos;dr,"If you offer suggestions to the service, they ...",True,"If you offer suggestions to the service, they ...",good,NaN,50,Generated through the annotate view,NaN,NaN,NaN,NaN,By contributing to this project (e.g.\nby send...


In [541]:
#random lookup to check data

df.loc[np.random.randint(1,2564,5),['quoteText','tldr','document']]

,quoteText,tldr,document
1482,Cookies are small data files that are commonly...,This is probably more of a functional issue th...,This is probably more of a functional issue th...
582,"As we continue to develop our business, we mig...",Amazon may sell user data as part of a busines...,Amazon may sell user data as part of a busines...
2174,"Don’t use any kind of software, device or meth...",Generated through the annotate view,"Don’t use any kind of software, device or meth..."
735,We use Google Analytics which is a web analyti...,Generated through the annotate view,We use Google Analytics which is a web analyti...
662,In some circumstances and to the extent permit...,Generated through the annotate view,In some circumstances and to the extent permit...


In [575]:
#the land of sad documents
df.document[0],df.document[722]

('The terms of service state that "Signal does not sell, rent or monetize your personal data or content in any way – ever."',
 'Last updated: June 07, 2019')

In [539]:
tfiddy = TfidfVectorizer()
tfiddy.fit(df['document'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [485]:
X = tfiddy.transform(df['document']).toarray()

In [486]:
X.shape

(2560, 5451)

In [574]:
'''uses a method in tfidf to explore the initial clean up step - lowering 
case, removing punctuation, and splitting words'''

tfiddy = TfidfVectorizer()
springcleaning = tfiddy.build_analyzer()
#before
print(df['document'][87])
#after
print(springcleaning(df['document'][87]))

"Qwant also collects connection data, not associated with your search queries, only to ensure the security of its information system and, if you create or modify Boards, to respect the obligation to identify content creators such as mandated by French law (article 6-II de la Loi pour la confiance en l’économie numérique)."
['qwant', 'also', 'collects', 'connection', 'data', 'not', 'associated', 'with', 'your', 'search', 'queries', 'only', 'to', 'ensure', 'the', 'security', 'of', 'its', 'information', 'system', 'and', 'if', 'you', 'create', 'or', 'modify', 'boards', 'to', 'respect', 'the', 'obligation', 'to', 'identify', 'content', 'creators', 'such', 'as', 'mandated', 'by', 'french', 'law', 'article', 'ii', 'de', 'la', 'loi', 'pour', 'la', 'confiance', 'en', 'économie', 'numérique']


In [549]:
#before
df['document'][88]

'"When you use Qwant, no personal information whatsoever is neither captured or transmitted to advertisers.\r\nIn details:\r\n– No third party cookies\r\n– ..."\r\n\r\nand \r\n\r\n"We don’t use any cookie nor any tracking device that may allow us to track your browsing habits or to establish your profile."'

In [545]:
#after
springcleaning(df['document'][88])

['when',
 'you',
 'use',
 'qwant',
 'no',
 'personal',
 'information',
 'whatsoever',
 'is',
 'neither',
 'captured',
 'or',
 'transmitted',
 'to',
 'advertisers',
 'in',
 'details',
 'no',
 'third',
 'party',
 'cookies',
 'and',
 'we',
 'don',
 'use',
 'any',
 'cookie',
 'nor',
 'any',
 'tracking',
 'device',
 'that',
 'may',
 'allow',
 'us',
 'to',
 'track',
 'your',
 'browsing',
 'habits',
 'or',
 'to',
 'establish',
 'your',
 'profile']